In [ ]:
# Install the required libraries
!pip install prophet

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
home_ownership_rate = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Home Ownership Rate (Hamilton, OH) - Home Ownership Rate.csv")
us_census = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/US Census (Hamilton, OH) - Sheet1.csv")
display(home_ownership_rate)
display(us_census)

,DATE,HOWNRATEACS039061
0,1/1/2009,67.089647
1,1/1/2010,65.820411
2,1/1/2011,65.362558
3,1/1/2012,64.354125
4,1/1/2013,63.293043
5,1/1/2014,62.709304
6,1/1/2015,62.118683
7,1/1/2016,61.708481
8,1/1/2017,61.738673
9,1/1/2018,62.062800


,Fact,Fact Note,"Hamilton County, Ohio","Value Note for Hamilton County, Ohio"
0,"Population estimates, July 1, 2023, (V2023)",NaN,NaN,NaN
1,"Population Estimates, July 1, 2022, (V2022)",NaN,"825,037",NaN
2,"Population estimates base, April 1, 2020, (V2023)",NaN,NaN,NaN
3,"Population estimates base, April 1, 2020, (V2022)",NaN,"830,625",NaN
4,"Population, percent change - April 1, 2020 (es...",NaN,NaN,NaN
...,...,...,...,...
83,FN,Footnote on this item in place of data,NaN,NaN
84,X,Not applicable,NaN,NaN
85,S,Suppressed; does not meet publication standards,NaN,NaN
86,NaN,Not available,NaN,NaN
